In [10]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt
import sys
import numpy as np
import pandas as pd


In [13]:
csv_File_Path = r'E:\A_Z Handwritten Data\A_Z Handwritten Data.csv'

df = pd.read_csv(csv_File_Path)

In [42]:
print(len(df))

print(df.columns)


372450
Index(['0', '0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9',
       ...
       '0.639', '0.640', '0.641', '0.642', '0.643', '0.644', '0.645', '0.646',
       '0.647', '0.648'],
      dtype='object', length=785)


In [121]:
X_train =df.loc[df['0'] == 0]
print(len(X_train))

# print(X_train.columns)
print(X_train.shape)
print(type(X_train))
# X_train.head(100)

13869
(13869, 785)
<class 'pandas.core.frame.DataFrame'>


In [122]:
X_train.drop(X_train.columns[[0]], axis=1, inplace=True, errors='ignore')

c:\users\sachin\appdata\local\programs\python\python36\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [123]:
X_train = X_train.as_matrix()
print(X_train.shape)
print(type(X_train))

(13869, 784)
<class 'numpy.ndarray'>


c:\users\sachin\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [124]:
X_train = X_train.reshape(len(X_train), 28,28)
print(X_train.shape)
print(type(X_train))

(13869, 28, 28)
<class 'numpy.ndarray'>


In [125]:
class GAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):
        global X_train

        # Load the dataset
#         (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%d.png" % epoch)
        plt.close()

In [126]:
if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=10000, batch_size=64, sample_interval=100)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_8 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_50 (Dense)             (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_36 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_51 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_37 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_52 (Dense)             (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________
____

c:\users\sachin\appdata\local\programs\python\python36\lib\site-packages\keras\engine\training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.583063, acc.: 69.53%] [G loss: 0.538758]
1 [D loss: 0.504295, acc.: 58.59%] [G loss: 0.612273]
2 [D loss: 0.391489, acc.: 72.66%] [G loss: 0.743884]
3 [D loss: 0.341186, acc.: 79.69%] [G loss: 0.921551]
4 [D loss: 0.276229, acc.: 94.53%] [G loss: 1.103401]
5 [D loss: 0.249730, acc.: 92.97%] [G loss: 1.233820]
6 [D loss: 0.194753, acc.: 99.22%] [G loss: 1.363058]
7 [D loss: 0.179362, acc.: 99.22%] [G loss: 1.478152]
8 [D loss: 0.153602, acc.: 100.00%] [G loss: 1.639258]
9 [D loss: 0.135973, acc.: 100.00%] [G loss: 1.662568]
10 [D loss: 0.123219, acc.: 99.22%] [G loss: 1.801864]
11 [D loss: 0.115328, acc.: 100.00%] [G loss: 1.886030]
12 [D loss: 0.105660, acc.: 100.00%] [G loss: 1.918738]
13 [D loss: 0.090109, acc.: 100.00%] [G loss: 2.036277]
14 [D loss: 0.086567, acc.: 100.00%] [G loss: 2.146991]
15 [D loss: 0.080137, acc.: 100.00%] [G loss: 2.217789]
16 [D loss: 0.070547, acc.: 100.00%] [G loss: 2.202156]
17 [D loss: 0.072073, acc.: 100.00%] [G loss: 2.252996]
18 [D loss:

147 [D loss: 0.129052, acc.: 97.66%] [G loss: 3.445051]
148 [D loss: 0.592845, acc.: 75.78%] [G loss: 2.286204]
149 [D loss: 0.220490, acc.: 88.28%] [G loss: 3.248943]
150 [D loss: 0.105075, acc.: 100.00%] [G loss: 3.657950]
151 [D loss: 0.293249, acc.: 87.50%] [G loss: 2.711998]
152 [D loss: 0.117229, acc.: 96.88%] [G loss: 3.723650]
153 [D loss: 0.412561, acc.: 82.03%] [G loss: 2.281950]
154 [D loss: 0.166099, acc.: 94.53%] [G loss: 3.612335]
155 [D loss: 0.195770, acc.: 94.53%] [G loss: 2.884840]
156 [D loss: 0.114502, acc.: 96.88%] [G loss: 3.204101]
157 [D loss: 0.181437, acc.: 93.75%] [G loss: 3.562797]
158 [D loss: 0.272147, acc.: 89.06%] [G loss: 3.408417]
159 [D loss: 0.200506, acc.: 94.53%] [G loss: 3.414757]
160 [D loss: 0.435585, acc.: 78.12%] [G loss: 2.877016]
161 [D loss: 0.208100, acc.: 91.41%] [G loss: 3.785098]
162 [D loss: 0.811273, acc.: 64.84%] [G loss: 1.896055]
163 [D loss: 0.362593, acc.: 80.47%] [G loss: 2.674569]
164 [D loss: 0.101812, acc.: 97.66%] [G loss: 4

295 [D loss: 0.679983, acc.: 46.88%] [G loss: 0.630290]
296 [D loss: 0.672795, acc.: 48.44%] [G loss: 0.620118]
297 [D loss: 0.674203, acc.: 49.22%] [G loss: 0.610543]
298 [D loss: 0.678012, acc.: 45.31%] [G loss: 0.639947]
299 [D loss: 0.680568, acc.: 46.09%] [G loss: 0.647262]
300 [D loss: 0.670615, acc.: 48.44%] [G loss: 0.659657]
301 [D loss: 0.686188, acc.: 46.09%] [G loss: 0.648879]
302 [D loss: 0.678735, acc.: 48.44%] [G loss: 0.639492]
303 [D loss: 0.662536, acc.: 50.00%] [G loss: 0.640881]
304 [D loss: 0.681162, acc.: 47.66%] [G loss: 0.652419]
305 [D loss: 0.661060, acc.: 50.00%] [G loss: 0.650992]
306 [D loss: 0.664876, acc.: 49.22%] [G loss: 0.662984]
307 [D loss: 0.671531, acc.: 49.22%] [G loss: 0.657785]
308 [D loss: 0.677780, acc.: 49.22%] [G loss: 0.649265]
309 [D loss: 0.668294, acc.: 50.00%] [G loss: 0.648115]
310 [D loss: 0.661554, acc.: 50.00%] [G loss: 0.647807]
311 [D loss: 0.659781, acc.: 47.66%] [G loss: 0.642882]
312 [D loss: 0.672187, acc.: 50.00%] [G loss: 0.

...

9416 [D loss: 0.659077, acc.: 60.16%] [G loss: 0.819754]
9417 [D loss: 0.625083, acc.: 65.62%] [G loss: 0.879889]
9418 [D loss: 0.699382, acc.: 50.00%] [G loss: 0.859605]
9419 [D loss: 0.691817, acc.: 52.34%] [G loss: 0.846217]
9420 [D loss: 0.630401, acc.: 66.41%] [G loss: 0.828237]
9421 [D loss: 0.656734, acc.: 59.38%] [G loss: 0.803852]
9422 [D loss: 0.686596, acc.: 51.56%] [G loss: 0.822419]
9423 [D loss: 0.673269, acc.: 58.59%] [G loss: 0.800350]
9424 [D loss: 0.687178, acc.: 53.91%] [G loss: 0.830420]
9425 [D loss: 0.677728, acc.: 54.69%] [G loss: 0.798293]
9426 [D loss: 0.634543, acc.: 67.97%] [G loss: 0.815666]
9427 [D loss: 0.712142, acc.: 50.00%] [G loss: 0.824649]
9428 [D loss: 0.670730, acc.: 60.16%] [G loss: 0.776837]
9429 [D loss: 0.715601, acc.: 46.09%] [G loss: 0.784614]
9430 [D loss: 0.673309, acc.: 59.38%] [G loss: 0.803497]
9431 [D loss: 0.684929, acc.: 55.47%] [G loss: 0.811417]
9432 [D loss: 0.657982, acc.: 58.59%] [G loss: 0.814217]
9433 [D loss: 0.649497, acc.: 6

9561 [D loss: 0.676325, acc.: 58.59%] [G loss: 0.813701]
9562 [D loss: 0.637251, acc.: 64.84%] [G loss: 0.800476]
9563 [D loss: 0.671142, acc.: 55.47%] [G loss: 0.824807]
9564 [D loss: 0.690472, acc.: 50.78%] [G loss: 0.837001]
9565 [D loss: 0.683237, acc.: 56.25%] [G loss: 0.845364]
9566 [D loss: 0.661198, acc.: 58.59%] [G loss: 0.851315]
9567 [D loss: 0.653030, acc.: 67.19%] [G loss: 0.851333]
9568 [D loss: 0.685799, acc.: 53.12%] [G loss: 0.831799]
9569 [D loss: 0.679178, acc.: 51.56%] [G loss: 0.845846]
9570 [D loss: 0.678518, acc.: 55.47%] [G loss: 0.821746]
9571 [D loss: 0.655610, acc.: 65.62%] [G loss: 0.822067]
9572 [D loss: 0.682899, acc.: 57.81%] [G loss: 0.773502]
9573 [D loss: 0.684283, acc.: 56.25%] [G loss: 0.798026]
9574 [D loss: 0.657210, acc.: 60.94%] [G loss: 0.829846]
9575 [D loss: 0.647254, acc.: 63.28%] [G loss: 0.833414]
9576 [D loss: 0.678311, acc.: 56.25%] [G loss: 0.857339]
9577 [D loss: 0.703597, acc.: 47.66%] [G loss: 0.820479]
9578 [D loss: 0.663545, acc.: 6

9705 [D loss: 0.664936, acc.: 60.94%] [G loss: 0.809703]
9706 [D loss: 0.701602, acc.: 59.38%] [G loss: 0.855430]
9707 [D loss: 0.684816, acc.: 56.25%] [G loss: 0.854473]
9708 [D loss: 0.662689, acc.: 60.16%] [G loss: 0.864277]
9709 [D loss: 0.688500, acc.: 49.22%] [G loss: 0.850876]
9710 [D loss: 0.669192, acc.: 63.28%] [G loss: 0.814040]
9711 [D loss: 0.693145, acc.: 55.47%] [G loss: 0.801769]
9712 [D loss: 0.658084, acc.: 59.38%] [G loss: 0.851520]
9713 [D loss: 0.683236, acc.: 53.12%] [G loss: 0.817153]
9714 [D loss: 0.669400, acc.: 60.94%] [G loss: 0.827155]
9715 [D loss: 0.684546, acc.: 52.34%] [G loss: 0.805183]
9716 [D loss: 0.663703, acc.: 62.50%] [G loss: 0.803288]
9717 [D loss: 0.671612, acc.: 62.50%] [G loss: 0.836320]
9718 [D loss: 0.662340, acc.: 62.50%] [G loss: 0.819424]
9719 [D loss: 0.665904, acc.: 59.38%] [G loss: 0.825856]
9720 [D loss: 0.685678, acc.: 51.56%] [G loss: 0.842041]
9721 [D loss: 0.682684, acc.: 55.47%] [G loss: 0.826133]
9722 [D loss: 0.661039, acc.: 5

9850 [D loss: 0.667611, acc.: 54.69%] [G loss: 0.818920]
9851 [D loss: 0.649235, acc.: 60.16%] [G loss: 0.835432]
9852 [D loss: 0.700604, acc.: 50.78%] [G loss: 0.800809]
9853 [D loss: 0.675525, acc.: 57.81%] [G loss: 0.820851]
9854 [D loss: 0.655643, acc.: 59.38%] [G loss: 0.827084]
9855 [D loss: 0.664867, acc.: 58.59%] [G loss: 0.812570]
9856 [D loss: 0.650374, acc.: 68.75%] [G loss: 0.790922]
9857 [D loss: 0.665355, acc.: 57.81%] [G loss: 0.826020]
9858 [D loss: 0.672545, acc.: 54.69%] [G loss: 0.827865]
9859 [D loss: 0.687902, acc.: 53.12%] [G loss: 0.817714]
9860 [D loss: 0.679472, acc.: 57.03%] [G loss: 0.848782]
9861 [D loss: 0.664416, acc.: 56.25%] [G loss: 0.810517]
9862 [D loss: 0.690690, acc.: 53.91%] [G loss: 0.837798]
9863 [D loss: 0.673728, acc.: 50.00%] [G loss: 0.827333]
9864 [D loss: 0.639632, acc.: 66.41%] [G loss: 0.797324]
9865 [D loss: 0.656220, acc.: 63.28%] [G loss: 0.806684]
9866 [D loss: 0.674340, acc.: 57.03%] [G loss: 0.789410]
9867 [D loss: 0.674298, acc.: 6

9994 [D loss: 0.647469, acc.: 64.06%] [G loss: 0.806176]
9995 [D loss: 0.679204, acc.: 60.94%] [G loss: 0.815681]
9996 [D loss: 0.688355, acc.: 58.59%] [G loss: 0.810470]
9997 [D loss: 0.677021, acc.: 60.16%] [G loss: 0.856608]
9998 [D loss: 0.655839, acc.: 64.06%] [G loss: 0.847044]
9999 [D loss: 0.647949, acc.: 62.50%] [G loss: 0.844665]
